# Preparación de datos

### Importar librerias

In [1]:
import pandas as pd
import numpy as np
import ast
import json

### Cargar dataset 'movies_dataset.csv'

In [2]:
data_movies = pd.read_csv('Datasets\movies_dataset.csv', encoding='utf-8')

C:\Users\Lenovo ThinkPad T440\AppData\Local\Temp\ipykernel_3780\3722730968.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_movies = pd.read_csv('Datasets\movies_dataset.csv', encoding='utf-8')


In [3]:
data_movies.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


### Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos

In [4]:
# Desanidar campos para unirlos al dataset
# Listas

def fetch_name(obj):
    if isinstance(obj, str) and '{' in obj:
        s = []
        for i in ast.literal_eval(obj):
            s.append(i['name'])
        return s

# Diccionarios


def fetch_name1(obj):
    if isinstance(obj, str) and '{' in obj:
        dic = ast.literal_eval(obj)
        return dic['name']

In [5]:
data_movies['genres'] = data_movies['genres'].apply(fetch_name)
data_movies['belongs_to_collection'] = data_movies['belongs_to_collection'].apply(
    fetch_name1)
data_movies['production_companies'] = data_movies['production_companies'].apply(
    fetch_name)
data_movies['production_countries'] = data_movies['production_countries'].apply(
    fetch_name)
data_movies['spoken_languages'] = data_movies['spoken_languages'].apply(
    fetch_name)

In [6]:
data_movies.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,Toy Story Collection,30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,[English],Released,NaN,Toy Story,False,7.7,5415.0
1,False,None,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,Grumpy Old Men Collection,0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [7]:
data_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4491 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 43024 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   33585 non-null  object 
 13  production_countries   39178 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

### Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0

In [8]:
# Reemplazar valores nulos en 'revenue' y 'budget' con 0

data_movies['revenue'] = data_movies['revenue'].fillna(0)
data_movies['budget'] = data_movies['budget'].fillna(0)

### Los valores nulos del campo release date deben eliminarse

In [9]:
# Eliminar filas con valores nulos en 'release_date'

data_movies = data_movies.dropna(subset=['release_date'])

### De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno

In [10]:
# Convertir la columna 'release_date' al formato de fecha
data_movies['release_date'] = pd.to_datetime(
    data_movies['release_date'], errors='coerce')

# Eliminar las filas con valores no válidos en la columna 'release_date'
data_movies = data_movies.dropna(subset=['release_date'])

In [11]:
data_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45376 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   adult                  45376 non-null  object        
 1   belongs_to_collection  4488 non-null   object        
 2   budget                 45376 non-null  object        
 3   genres                 42992 non-null  object        
 4   homepage               7766 non-null   object        
 5   id                     45376 non-null  object        
 6   imdb_id                45362 non-null  object        
 7   original_language      45365 non-null  object        
 8   original_title         45376 non-null  object        
 9   overview               44435 non-null  object        
 10  popularity             45376 non-null  object        
 11  poster_path            45037 non-null  object        
 12  production_companies   33580 non-null  object        
 13  p

In [12]:
# Crear una nueva columna 'release_year' con el año de la columna 'release_date'
data_movies['release_year'] = data_movies['release_date'].dt.year

In [13]:
data_movies.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,release_year
0,False,Toy Story Collection,30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,[English],Released,NaN,Toy Story,False,7.7,5415.0,1995


### Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0

In [14]:
# Convertir la columna 'budget' a tipo float
data_movies['budget'] = data_movies['budget'].astype(float)

# Calcular la columna "return" con la división de "revenue" entre "budget", teniendo en cuenta los casos nulos y 0
data_movies['return'] = data_movies.apply(
    lambda row: row['revenue'] / row['budget'] if row['revenue'] and row['budget'] else 0, axis=1)

In [15]:
data_movies.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,release_year,return
0,False,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,[English],Released,NaN,Toy Story,False,7.7,5415.0,1995,12.451801


### Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [16]:
# Eliminar columnas no utilizadas
columns_to_drop = ['video', 'imdb_id', 'adult',
                   'original_title', 'poster_path', 'homepage']
data_movies = data_movies.drop(columns=columns_to_drop)

In [17]:
data_movies.head(1)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return
0,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,[English],Released,NaN,Toy Story,7.7,5415.0,1995,12.451801


### Cargar dataset 'credits.csv'

In [18]:
data_credits = pd.read_csv('Datasets\credits.csv', encoding='utf-8')

In [19]:
data_credits.head(1)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862


In [20]:
data_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


### Trabajar con 'cast' y 'crew'

In [21]:
# Función para convertir de diccionario a string
def parse_dict_string(value):
    if pd.isna(value):
        return []
    else:
        return ast.literal_eval(value)

In [22]:
# Aplicar la función para pasar de diccionario a string para luego poder desanidar
data_credits["cast"] = data_credits["cast"].apply(parse_dict_string)
data_credits["crew"] = data_credits["crew"].apply(parse_dict_string)

In [23]:
# Función para extraer los nombres de los actores
data_credits['names_actors'] = data_credits['cast'].apply(
    lambda registros: [registro['name'] for registro in registros])

# Función para extraer el nombre del director
data_credits['name_director'] = data_credits['crew'].apply(lambda registros: next(
    (registro['name'] for registro in registros if registro['job'] == 'Director'), None))

In [24]:
data_credits.head(1)

,cast,crew,id,names_actors,name_director
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter


In [25]:
# Seleccionar y crear un nuevo DataFrame con las columnas necesarias
columnas_necesarias = ["id", "names_actors", "name_director"]

# Nuevo DataFrame con las columnas necesarias
data_credits_cn = data_credits[columnas_necesarias]

In [26]:
data_credits_cn.head(1)

,id,names_actors,name_director
0,862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter


### Concatenar los datasets 'data_movies' y 'data_credits_cn' en uno solo

In [27]:
# Convertir la columna 'id' de tipo str a int64 en el DataFrame 'data_movies'
data_movies['id'] = data_movies['id'].astype('int64')

# Combinar los DataFrames en función de la columna 'id'
data_final = pd.merge(data_movies, data_credits_cn, on='id', how='inner')

In [28]:
data_final.head(1)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return,names_actors,name_director
0,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,...,[English],Released,NaN,Toy Story,7.7,5415.0,1995,12.451801,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter


In [29]:
data_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45451 entries, 0 to 45450
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4497 non-null   object        
 1   budget                 45451 non-null  float64       
 2   genres                 43067 non-null  object        
 3   id                     45451 non-null  int64         
 4   original_language      45440 non-null  object        
 5   overview               44510 non-null  object        
 6   popularity             45451 non-null  object        
 7   production_companies   33634 non-null  object        
 8   production_countries   39232 non-null  object        
 9   release_date           45451 non-null  datetime64[ns]
 10  revenue                45451 non-null  float64       
 11  runtime                45205 non-null  float64       
 12  spoken_languages       41677 non-null  object        
 13  s

### Limpiar 'data_final' para eliminar valores nulos

In [30]:
# Convertir columna de tipo objeto a numérico
data_final['popularity'] = pd.to_numeric(
    data_final['popularity'], errors='coerce')

# Imputar valores nulos
# Imputar valores nulos en 'genres' con una cadena vacía
data_final['genres'].fillna('', inplace=True)
# Imputar valores nulos en 'overview' con una cadena vacía
data_final['overview'].fillna('', inplace=True)
# Imputar valores nulos en 'name_director' con 'Unknown'
data_final['name_director'].fillna('Unknown', inplace=True)
# Imputar valores nulos en 'production_companies' con 'Unknown'
data_final['production_companies'].fillna('Unknown', inplace=True)
# Imputar valores nulos en 'production_countries' con 'Unknown'
data_final['production_countries'].fillna('Unknown', inplace=True)
# Imputar valores nulos en 'spoken_languages' con 'Unknown'
data_final['spoken_languages'].fillna('Unknown', inplace=True)

# Eliminar columnas con demasiados valores nulos que no son esenciales
data_final.drop(columns=['belongs_to_collection', 'tagline'], inplace=True)

# Eliminar filas con valores nulos en columnas críticas
data_final.dropna(subset=['original_language'], inplace=True)

# Imputar valores nulos en columnas numéricas
data_final['runtime'].fillna(data_final['runtime'].median(), inplace=True)

# Imputar valores nulos en columnas categóricas
data_final['status'].fillna('Unknown', inplace=True)

In [31]:
data_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45440 entries, 0 to 45450
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   budget                45440 non-null  float64       
 1   genres                45440 non-null  object        
 2   id                    45440 non-null  int64         
 3   original_language     45440 non-null  object        
 4   overview              45440 non-null  object        
 5   popularity            45440 non-null  float64       
 6   production_companies  45440 non-null  object        
 7   production_countries  45440 non-null  object        
 8   release_date          45440 non-null  datetime64[ns]
 9   revenue               45440 non-null  float64       
 10  runtime               45440 non-null  float64       
 11  spoken_languages      45440 non-null  object        
 12  status                45440 non-null  object        
 13  title           

In [32]:
data_final.head(1)

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count,release_year,return,names_actors,name_director
0,30000000.0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,[English],Released,Toy Story,7.7,5415.0,1995,12.451801,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter


### Simplificar data para modelo de ML

In [33]:
# Seleccionar las columnas relevantes para el modelo de ML
data_final_ML = data_final[[
    "title", "release_date", "release_year",
    "popularity", "vote_count", "vote_average", "names_actors",
    "name_director", "budget", "revenue", "return"
]]

# Filtrar películas lanzadas entre 1980 y 2020
data_final_ML = data_final_ML[
    (data_final_ML['release_year'] >= 1980) & (
        data_final_ML['release_year'] <= 2020)
]

# Filtrar películas con un promedio de votos mayor a 0
data_final_ML = data_final_ML[data_final_ML['vote_average'] > 0]

# Filtrar películas con un número de votos mayor a 10
data_final_ML = data_final_ML[data_final_ML['vote_count'] > 10]

# Filtrar películas con un budget mayor a 0
data_final_ML = data_final_ML[data_final_ML['budget'] > 0]

# Eliminar duplicados basados en la columna 'title'
data_final_ML = data_final_ML.drop_duplicates(subset='title')

# Tomar una muestra de 5000 registros
data_final_ML = data_final_ML.sample(n=5000, random_state=42)

### Archivo final para el modelo: 'data_final_ML.csv'

In [34]:
import os

In [35]:
# Especificar la carpeta de destino
output_folder = '../API'  # ../ para subir un nivel desde ETL a la raíz del proyecto
output_file = 'data_final_ML.csv'

# Crea la carpeta si no existe
os.makedirs(output_folder, exist_ok=True)

# Guarda el DataFrame en la carpeta de destino
data_final_ML.to_csv(os.path.join(output_folder, output_file), index=False)